In this tutorial, we'll take a look at how CrypTen performs inference with an encrypted neural network on encrypted data. We'll see how the data remains encrypted through all the operations, and yet is able to obtain accurate results after the computation. 


### A Simple Linear Layer
We'll start by examining how a single Linear layer works in CrypTen. We'll instantiate a Linear layer, encrypt it, and step through some toy data with it. We'll assume Alice has the layer (and the `rank` 0 process) and Bob has the data (and the `rank` 1 process).

<small><i>(Technical note: Because Jupyter notebooks run only a single process, we simulate a multi-party world with the @mpc.run_multiprocess decorator.)</i></small>

In [1]:
import crypten
import torch

crypten.init() 

#generate some toy data
features = 4
examples = 3
toy_data = torch.rand(examples, features)

In [2]:
#Instantiate single Linear layer
layer_linear = crypten.nn.Linear(4, 2)

#The weights and the bias are initialized to small random values
print("Plaintext Weights:", layer_linear._parameters['weight'])
print("Plaintext Bias:", layer_linear._parameters['bias'])

Plaintext Weights: Parameter containing:
tensor([[-0.4408, -0.1707,  0.0428, -0.2996],
        [-0.0617, -0.3127, -0.4000,  0.1824]], requires_grad=True)
Plaintext Bias: Parameter containing:
tensor([0.1103, 0.0856], requires_grad=True)


In [3]:
import crypten.mpc as mpc
import crypten.communicator as comm

@mpc.run_multiprocess(world_size=2)
def forward_single_encrypted_layer():
    rank = comm.get().get_rank()
    
    #Loading the layer
    if rank == 0:
        #Alice loads the layer
        layer_enc = layer_linear
    else:
        #Bob loads a dummy layer
        layer_enc = crypten.nn.Linear(4, 2)
        
    #Loading the toy data
    if rank == 1:
        #Bob loads the toy data
        toy_data_upd = toy_data
    else:
        #Alice loads dummy data
        toy_data_upd = torch.empty(toy_data.size())
    
    #Encrypting the linear layer and the data
    layer_enc.encrypt(src=0) #note src=0!
    toy_data_enc = crypten.cryptensor(toy_data_upd, src=1) #note src=1!
    
    #Let's now examine the shares inside the encrypted toy data
    #We only print rank 0 information for readability
    if rank == 1:
        print("Encrypted data:\n{}\n".format(toy_data_enc.share))

    #Let's also examine the weights and the bias of the linear layer again
    #We only print rank 0 information for readability
    if rank == 0:
        #First, we'll see that weights and bias have become encrypted tensors
        print("Weights Encrypted: {}".format(crypten.is_encrypted_tensor(layer_enc._parameters['weight'])))
        print("Bias Encrypted: {}".format(crypten.is_encrypted_tensor(layer_enc._parameters['bias'])))

        #Now let's look at the tensor values
        print("Encrypted Weights:\n{}".format(layer_enc._parameters['weight'].share))
        print("Encrypted Bias: {}".format(layer_enc._parameters['bias'].share))
        print()

    #apply the encrypted layer: encrypted linear transformation 
    result_enc = layer_enc.forward(toy_data_enc)
    
    #we'll print the resulting shares of both parties
    print("Rank:{} Encrypted result:\n{}\n".format(rank, result_enc.share))
    
    #decrypt the result:
    result_plaintext = result_enc.get_plain_text()
    #we'll print only rank 0 values for readability
    if rank == 0:
        print("Decrypted result:\n", result_plaintext)
        
z = forward_single_encrypted_layer()

Weights Encrypted: True
Encrypted data:
tensor([[-3529437650444850327, -1384961456004736779, -6929198735789959786,
          7050025713128459905],
        [-1628411347735222774, -1816930511366182472, -3256300204250465506,
          4493608843179679712],
        [ 2992598500924676096,  1437511533294625892,    85303661367237954,
          2971990899418421934]])
Bias Encrypted: True

Encrypted Weights:
tensor([[-4914577077683699796, -2413527856795080675,  2278954485404849699,
          3640852080763557905],
        [ 6133062412410498540,   257865353531543276, -4796330528663584032,
          8781451695104091633]])
Encrypted Bias: tensor([-6544850209041378305, -4417009090129027140])

Rank:0 Encrypted result:
tensor([[-6544782381304935970, -4417103682313702340],
        [-6544979681819059998, -4417126410741752232],
        [-6544985919889404546, -4416897672215962903]])
Rank:1 Encrypted result:
tensor([[6544782381304904285, 4417103682313676136],
        [6544979681819037111, 44171264107417125

We can see that the application of the encrypted linear layer on the encrypted data produces an encrypted result, which we can then decrypt to get the values in plaintext.

Let's look at a second linear transformation, to give a flavor of how accuracy is preserved even when the data and the layer are encrypted. We'll look at a uniform scaling transformation, in which all tensor elements are multiplied by the same scalar factor. Again, we'll assume Alice has the layer and the `rank` 0 process, and Bob has the data and the `rank` 1 process.

In [4]:
#Initialize a linear layer with random weights
layer_scale = crypten.nn.Linear(3, 3)

#Construct a uniform scaling matrix: we'll scale by factor 5
factor = 5
layer_scale._parameters['weight'] = torch.eye(3)*factor
layer_scale._parameters['bias'] = torch.zeros_like(layer_scale._parameters['bias'])

 #Construct some toy data
toy_data = torch.ones(2, 3)

@mpc.run_multiprocess(world_size=2)
def forward_scaling_layer():
    rank = comm.get().get_rank()
    
    if rank == 0:
        #Alice gets the scaling layer
        layer_enc = layer_scale
    else:
        #Bob gets a dummy layer
        layer_enc = layer_scale
    
    if rank == 1:
        #Bob gets the toy data
        data = toy_data
    else:
        #Alice gets dummy data
        data = torch.empty(toy_data.size())
    
    
    #Encrypt the layer
    layer_enc.encrypt(src=0)
    #Encrypt the toy data
    toy_data_enc = crypten.cryptensor(data, src=1)
    
    #Let's first look at the encrypted data
    #We'll restrict printing to rank 1 for readability
    if rank == 1:
        print("Encrypted data:\n{}\n".format(toy_data_enc.share))
        
    #Let's now look inside the encrypted layer
    #We'll restrict printing to rank 0 for readability
    if rank == 0:
        print("Encrypted Weights:\n{}".format(layer_enc._parameters['weight'].share))
        print("Encrypted Bias:\n{}".format(layer_enc._parameters['bias'].share))

    #Apply the encrypted scaling transformation
    result_enc = layer_enc.forward(toy_data_enc)
    
    #Let's examine the encrypted results
    print("Rank {} Encrypted result:\n {}\n".format(rank, result_enc.share))

    #Decrypt the result:
    result_plaintext = result_enc.get_plain_text()
    #We'll print only rank 0 for readability
    if rank == 0:
        print("Plaintext result:\n{}".format(result_plaintext))
        
z = forward_scaling_layer()

Encrypted data:
tensor([[-5274762416354429527,  3044720233427754283, -8084390380562988153],
        [ 4844508570170404561,  -739434560346899506,  9015698232244756031]])
Encrypted Weights:
tensor([[  883198664398464434,  1303801953729869595,  9032040305735239635],
        [  607235406868532498,  3299040990922350992,   538347453982252238],
        [ 2915046868926881237,  1765978872310897706, -2863619390377126375]])

Encrypted Bias:
tensor([-7901549011774995946,  -937043077034743511, -7669741716401089189])
Rank 0 Encrypted result:
 tensor([[-7901487379265356323,  -937179007628379228, -7669839616455256762],
        [-7901657443855509253,  -937015236487772829, -7669760671194908171]])
Rank 1 Encrypted result:
 tensor([[7901487379265684003,  937179007628706908, 7669839616455584442],
        [7901657443855836933,  937015236488100509, 7669760671195235851]])


Plaintext result:
tensor([[5., 5., 5.],
        [5., 5., 5.]])


The resulting plaintext tensor is correctly scaled, even though we applied the encrypted transformation on the encrypted input! 

### Multi-layer Neural Networks
Let's now look at how the encrypted input moves through an encrypted multi-layer neural network. 

For ease of explanation, we'll first step through a network with only two linear layers and ReLU activations. Again, we'll assume Alice has a network and Bob has some data, and they wish to run encrypted inference. 

To simulate this, we'll once again generate some toy data and train Alice's network on it. Then we'll encrypt Alice's network, Bob's data, and step through every layer in the network with the encrypted data. Through this, we'll see how the computations get applied although the network and the data are encrypted.

In [5]:
# We'll generate some random data for illustration purposes.
features = 50
examples = 100
data = torch.randn(examples, features)
w_true = torch.randn(1, features)
b_true = torch.randn(1)
y = w_true.matmul(data.t()) + b_true
y = y.sign()

#change labels to format needed by training
y2 = torch.where(y==-1, torch.zeros(y.size()), y)
y2 = y2.squeeze().long()

In [6]:
#Define Alice and Bob's data
data_alice = data[:90,:]
data_bob = data[90:,:]

label_alice = y2[:90]
label_bob = y2[90:]

In [7]:
#Alice creates a very simple one layer network
import torch.nn as nn
import torch.nn.functional as F

#Define Alice's network
class AliceNet(nn.Module):
    def __init__(self):
        super(AliceNet, self).__init__()
        self.fc1 = nn.Linear(50, 20)
        self.fc2 = nn.Linear(20, 2)
        
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

model = AliceNet()

#Train Alice's network
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for i in range(500):  
    #forward pass: compute prediction
    output = model(data_alice)
    
    #compute and print loss
    loss = criterion(output, label_alice)
    if i % 100 == 99:
        print("Epoch", i, "Loss:", loss.item())
    
    #zero gradients for learnable parameters
    optimizer.zero_grad()
    
    #backward pass: compute gradient with respect to model parameters
    loss.backward()
    
    #update model parameters
    optimizer.step()

Epoch 99 Loss: 0.07619381695985794
Epoch 199 Loss: 0.01975707896053791
Epoch 299 Loss: 0.009856403805315495
Epoch 399 Loss: 0.006247421260923147
Epoch 499 Loss: 0.0044706217013299465


Let's now look at what happens when we encrypt the network that Alice has trained. (For readability, we illustrate this in a world with only Alice, and do not simulate multi-party world or include the code for Bob's process).

In [8]:
#Let's look at what happens when a neural network gets encrypted

#Create dummy input of the correct input shape for the model
dummy_input = torch.empty((1, 50))

#Encrypt the network
private_model = crypten.nn.from_pytorch(model, dummy_input)
private_model.encrypt(src=0)

#Let's look at the structure of the encrypted network
for name, curr_module in private_model._modules.items():
    print("Name:", name, "\tModule:", curr_module)

Name: 5 	Module: <crypten.nn.module.Linear object at 0x7fe0d8d7eac8>
Name: 6 	Module: <crypten.nn.module.ReLU object at 0x7fe05b8d55c0>
Name: output 	Module: <crypten.nn.module.Linear object at 0x7fe05b8d55f8>


We see that the encrypted networks has 3 modules, named '5', '6' and 'output', denoting the first Linear layer, the ReLU activation, and the second Linear layer respectively. These modules are encrypted just as the layers in the previous section were. 

Now let's encrypt Bob's data, and step it through each encrypted module. For readability, we will use only 3 examples from Bob's data to illustrate the inference. Note how Bob's data remains encrypted after each individual layer's computation!

In [9]:
#Select only the first three examples of Bob's data for readability
data = data_bob[:3,:]

#Create dummy input of the correct input shape for the model
dummy_input = torch.empty((1, 50))

@mpc.run_multiprocess(world_size=2)
def step_through_two_layers():
    
    rank = comm.get().get_rank()
    
    if rank == 0:
        #Alice gets the trained network
        model_upd = model
    else:
        #Bob gets a dummy layer
        model_upd = AliceNet()
        
    if rank == 1:
        #Bob gets the trained data
        data_upd = data
    else:
        #Alice gets the dummy layer
        data_upd = torch.empty(data.size())

    #Encrypt the network
    private_model = crypten.nn.from_pytorch(model_upd, dummy_input)
    private_model.encrypt(src=0)

    #Encrypt the data
    data_enc = crypten.cryptensor(data, src=1)

    #forward through the first layer
    out_enc = private_model._modules['5'].forward(data_enc)
    print("Rank: {} First Linear Layer: Output Encrypted: {}\n".format(rank, crypten.is_encrypted_tensor(out_enc)))
    print("Rank: {} Shares after First Linear Layer:{}\n".format(rank, out_enc.share))

    #apply ReLU activation
    out_enc = private_model._modules['6'].forward(out_enc)
    print("Rank: {} ReLU:\n Output Encrypted: {}\n".format(rank, crypten.is_encrypted_tensor(out_enc)))
    print("Rank: {} Shares after ReLU: {}\n".format(rank, out_enc.share))

    #forward through the second Linear layer
    out_enc = private_model._modules['output'].forward(out_enc)
    print("Rank: {} Second Linear layer:\n Output Encrypted: {}\n".format(rank, crypten.is_encrypted_tensor(out_enc))), 
    print("Rank: {} Shares after Second Linear layer:{}\n".format(rank, out_enc.share))

    #decrypt the output
    out_dec = out_enc.get_plain_text()
    #For readability, only print the rank 0 output (identical for rank 1)
    if rank == 0:
        print("Decrypted output:\n Output Encrypted:", crypten.is_encrypted_tensor(out_dec))
        print("Tensors:\n", out_dec)
    
z = step_through_two_layers()

Rank: 0 First Linear Layer: Output Encrypted: True
Rank: 1 First Linear Layer: Output Encrypted: True


Rank: 1 Shares after First Linear Layer:tensor([[-7903544318455955739,  7617702702855408976, -7052570795069168591,
          1727656015794399854,  5159531949446533624,  7228455143419277603,
          5785954513225743731, -2125448590957328485,  4798305163009888226,
          4469307498453606896,   611300775826289751,  6608534528173682827,
          1842994992776239801,  5773411982157767484,  2309732521878634774,
         -4195880852394788682,  7373027241720826123, -4490330287736089950,
          3025915678758970324, -9115159967454467117],
        [-7903493116439411434,  7617774502227285949, -7052422176647185589,
          1727779565068718304,  5159715460184703137,  7228591328842862996,
          5785957688218679039, -2125412666572421222,  4798439219769935731,
          4469312955166640597,   611429997679430363,  6608583579100475340,
          1842813703491054238,  5773363991466985580,

Again, we emphasize that the output of each layer is an encrypted tensor. Only after the final call to `get_plain_text` do we get the plaintext tensor.

We have used a simple two-layer network in our example, but the same ideas apply to more complex networks and operations. However, in more complex networks, there may not always be a one-to-one mapping between the PyTorch layers and the CrypTen layers. As an example, we'll take a typical network used to classify digits in MNIST data, and look at what happens to its structure when encrypted. (As we only wish to illustrate the structural changes in layers, we will not train this network on data; we will just use it with its randomly initialized weights). 

In [29]:
#Define Alice's network
class AliceNet2(nn.Module):
    def __init__(self):
        super(AliceNet2, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=0)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 4 * 4, 100)
        self.fc2 = nn.Linear(100, 10)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.batchnorm3 = nn.BatchNorm1d(100)
 
    def forward(self, x):
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = F.relu(out)
        out = F.avg_pool2d(out, 2)
        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = F.relu(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.batchnorm3(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out
    
model = AliceNet2()

#Let's encrypt the more complex network. 
#Create dummy input of the correct input shape for the model
dummy_input = torch.empty((1, 1, 28, 28))

#Encrypt the network
private_model = crypten.nn.from_pytorch(model, dummy_input)
private_model.encrypt(src=0)

#Let's look at the structure of the encrypted network
for name, curr_module in private_model._modules.items():
    print("Name:", name, "\tModule:", curr_module)

Name: 24 	Module: <crypten.nn.module.Conv2d object at 0x7fe0402f6908>
Name: 25 	Module: <crypten.nn.module._BatchNorm object at 0x7fe0402f6358>
Name: 26 	Module: <crypten.nn.module.ReLU object at 0x7fe0402f67b8>
Name: 27 	Module: <crypten.nn.module._ConstantPad object at 0x7fe0402f6048>
Name: 28 	Module: <crypten.nn.module.AvgPool2d object at 0x7fe0402f62e8>
Name: 29 	Module: <crypten.nn.module.Conv2d object at 0x7fe0402f6860>
Name: 30 	Module: <crypten.nn.module._BatchNorm object at 0x7fe0402f6cc0>
Name: 31 	Module: <crypten.nn.module.ReLU object at 0x7fe0402f6a58>
Name: 32 	Module: <crypten.nn.module._ConstantPad object at 0x7fe0402f6be0>
Name: 33 	Module: <crypten.nn.module.AvgPool2d object at 0x7fe0402f6d68>
Name: 34 	Module: <crypten.nn.module.Constant object at 0x7fe0402e6358>
Name: 35 	Module: <crypten.nn.module.Shape object at 0x7fe0402e60f0>
Name: 36 	Module: <crypten.nn.module.Gather object at 0x7fe0402e6198>
Name: 37 	Module: <crypten.nn.module.Constant object at 0x7fe0402e6

Notice how the CrypTen network has split the PyTorch network into several additional layers. Each PyTorch operation may correspond to one or more operations in CrypTen. Nevertheless, the same ideas apply when forwarding the data through the CrypTen equivalent of each PyTorch operation. 

For additional details, please see the CrypTen whitepaper. 